In [2]:

# !pip install opencv-python

In [6]:
import os
import cv2
import re
import json
imgs = os.listdir(r".\ECCV_data\data\imgs")
jsons = os.listdir(r".\ECCV_data\data\jsons")

files = []

for name in imgs:
    filename, _ = os.path.splitext(name)
    files.append(filename)
print(f'Folder contains {len(files)} images.')

Folder contains 300 images.


In [4]:

img_bbox_dir = os.path.join(r".\ECCV_data\data", "img_bbox")

if not os.path.exists(img_bbox_dir):
    os.makedirs(img_bbox_dir)

yolo_annotation_dir = os.path.join(r".\ECCV_data\data", "yolo_annotations")

if not os.path.exists(yolo_annotation_dir):
    os.makedirs(yolo_annotation_dir)

for file in files:
    img_path = os.path.join(r".\ECCV_data\data\imgs", file + ".png")
    json_path = os.path.join(r".\ECCV_data\data\jsons", file + ".json")
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)  

    with open(json_path, 'r') as f:
        data = json.load(f)

    yolo_annotations = set()  

    for widget_type in data:
        widgets = data[widget_type]
        for widget in widgets:
            jsonClass = widget.get('jsonClass')

            if jsonClass == 'ChoiceField':
                x = int(widget['x'])
                y = int(widget['y'])
                w = int(widget['w'])
                h = int(widget['h'])

                img_h, img_w, _ = image.shape
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                width = w / img_w
                height = h / img_h

                yolo_annotation = f"0 {x_center} {y_center} {width} {height}"

                if yolo_annotation not in yolo_annotations:
                    yolo_annotations.add(yolo_annotation)

                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, jsonClass, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if jsonClass == 'Widget':
                x = int(widget['x'])
                y = int(widget['y'])
                w = int(widget['w'])
                h = int(widget['h'])

                if  abs(w-h)<=7:
                    # print(w,h)
                    img_h, img_w, _ = image.shape
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    width = w / img_w
                    height = h / img_h

                    yolo_annotation = f"0 {x_center} {y_center} {width} {height}"

                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, jsonClass, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    out_bbox_filename = os.path.join(img_bbox_dir, file + "_bbox.png")
    cv2.imwrite(out_bbox_filename, image)

    out_yolo_filename = os.path.join(yolo_annotation_dir, file + ".txt")
    with open(out_yolo_filename, 'w') as yolo_file:
        for yolo_annotation in yolo_annotations:
            yolo_file.write(yolo_annotation + '\n')

cv2.destroyAllWindows()


In [5]:
# Define the output directory for images with bounding boxes
img_bbox_dir_widget = os.path.join(r".\ECCV_data\data", "img_bbox_wid")

# Create the output directory if it doesn't exist
if not os.path.exists(img_bbox_dir_widget):
    os.makedirs(img_bbox_dir_widget)

# Define the output directory for YOLO annotations
yolo_annotation_dir = os.path.join(r".\ECCV_data\data", "yolo_annotations")

# Create the output directory if it doesn't exist
if not os.path.exists(yolo_annotation_dir):
    os.makedirs(yolo_annotation_dir)

for file in files:
    img_path = os.path.join(r".\ECCV_data\data\imgs", file + ".png")
    json_path = os.path.join(r".\ECCV_data\data\jsons", file + ".json")
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read image in color mode

    with open(json_path, 'r') as f:
        data = json.load(f)

    yolo_annotations = set()  # Use a set to store unique YOLO annotations

    for widget_type in data:
        widgets = data[widget_type]
        for widget in widgets:
            jsonClass = widget.get('jsonClass')

        
            if jsonClass == 'Widget':
                x = int(widget['x'])
                y = int(widget['y'])
                w = int(widget['w'])
                h = int(widget['h'])

                if  abs(w-h)<=15:
                    # print(w,h)
                # Convert coordinates to YOLO format
                    img_h, img_w, _ = image.shape
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    width = w / img_w
                    height = h / img_h

                    yolo_annotation = f"0 {x_center} {y_center} {width} {height}"

                    # Check if the annotation is not already processed
                    if yolo_annotation not in yolo_annotations:
                        yolo_annotations.add(yolo_annotation)

                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, jsonClass, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    out_bbox_filename = os.path.join(img_bbox_dir_widget, file + "_bbox.png")
    cv2.imwrite(out_bbox_filename, image)

    out_yolo_filename = os.path.join(yolo_annotation_dir, file + ".txt")
    with open(out_yolo_filename, 'w') as yolo_file:
        for yolo_annotation in yolo_annotations:
            yolo_file.write(yolo_annotation + '\n')

cv2.destroyAllWindows()